Importing necessary libraries

In [59]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

Getting html file from given url and storing it locally using beautiful soap library

In [2]:
htmlData  = requests.get( "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soupData = BeautifulSoup(htmlData.text, 'html.parser')

Initializing the data frame

In [3]:
tableData = pd.DataFrame(columns=['PostalCode','Borough','Neighbourhood'])

Finding table rows from html and looping through and adding data in our dataframe

In [4]:
table_rows = soupData.table.find_all('tr')
table_rows.pop(0)

for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    tableData=tableData.append({'PostalCode':row[0],'Borough':row[1],'Neighbourhood':row[2].replace('\n','')}, ignore_index=True)

Checking data inserted successfully

In [5]:
tableData.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Removing rows where Borough is not assigned

In [6]:
tableData = tableData[tableData.Borough != 'Not assigned']
tableData.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Merging neighbourhoods based on Boroughs and Postal Code

In [7]:
tableData=tableData.groupby(['PostalCode', 'Borough']).apply(lambda group: ','.join(group['Neighbourhood']))
tableData=tableData.reset_index()
tableData.columns=['PostalCode','Borough','Neighbourhood']
tableData.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Assigning Borough value to Neighbourhood where Neighbourhood is not assigned

In [8]:
tableData.loc[tableData.Neighbourhood == 'Not assigned','Neighbourhood']= tableData.loc[tableData.Neighbourhood == 'Not assigned','Borough']

Printing shape for final dataframe

In [9]:
 tableData.shape

(103, 3)

In [11]:
!conda install -c conda-forge geocoder --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

    geocoder:        1.38.1-py_1       conda-forge
    ratelim:         0.1.6-py_2        conda-forge

The following packages will be UPDATED:

    

Importing libraries to get long lat values

In [12]:
import geocoder # import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Getting long lat values using geolocator for neighbourhoods and storing in lists

In [13]:

geolocator = Nominatim(user_agent="geoapiExercises")
long=[]
lat=[]


for nghbr in tableData.Neighbourhood:
    org_nghbr=nghbr
    print(org_nghbr)
    if nghbr.find(',') is not None:
        if nghbr.find(',')>0:
            nghbr=nghbr[0:nghbr.find(',')]
    location = geolocator.geocode(nghbr)
    if location is not None:
        latitude = location.latitude
        longitude = location.longitude
    else:
        nghbr=org_nghbr[org_nghbr.find(',')+1:len(org_nghbr)]
        location = geolocator.geocode(nghbr)
        if location is not None:
            latitude = location.latitude
            longitude = location.longitude
        else:
            latitude=0
            longitude=0
    long.append(longitude)
    lat.append(latitude)

print('lists generated')

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

Merging long lat values in dataframe

In [14]:
tableData['latitude_geocode']=lat

In [15]:
tableData['longitude_geocode']=long

In [16]:
tableData.head()

,PostalCode,Borough,Neighbourhood,latitude_geocode,longitude_geocode
0,M1B,Scarborough,"Rouge,Malvern",48.283710,0.705539
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.790117,-79.173334
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.755225,-79.198229
3,M1G,Scarborough,Woburn,42.479262,-71.152277
4,M1H,Scarborough,Cedarbrae,50.956318,-114.129323


Getting values from csv files because geolocator values had issues

In [17]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_3b5dbc687f174104a1309c870960e45e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='6W6g517W9kskKgP1LblhpXiRF4fngzeb30PmBoMI022M',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_3b5dbc687f174104a1309c870960e45e.get_object(Bucket='capstoneproject-donotdelete-pr-zgxcajzxmpngov',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging two dataframes to get long lat values in with neighbourhoods in one dataframe

In [18]:
df_data_1.set_index('Postal Code',inplace=True)

tableData['Postal Code']=tableData['PostalCode']
tableData.set_index('Postal Code',inplace=True)
tableData.drop("PostalCode", axis=1,inplace=True)
geoData = pd.concat([tableData,df_data_1], axis=1, sort=False)
geoData.reset_index(inplace=True)


In [20]:
geoData.head()

,Postal Code,Borough,Neighbourhood,latitude_geocode,longitude_geocode,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",48.283710,0.705539,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.790117,-79.173334,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.755225,-79.198229,43.763573,-79.188711
3,M1G,Scarborough,Woburn,42.479262,-71.152277,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,50.956318,-114.129323,43.773136,-79.239476


Filtering neighbourhoods of toronto

In [21]:
torontoData=geoData[geoData['Borough'].str.contains("Toronto")]

In [22]:
torontoData.head()

,Postal Code,Borough,Neighbourhood,latitude_geocode,longitude_geocode,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.671024,-79.296712,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",40.820312,-87.977818,43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.671024,-79.296712,43.668999,-79.315572
43,M4M,East Toronto,Studio District,-12.271035,49.291897,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,42.152279,-80.023112,43.728020,-79.388790


Defining foursquare credentials, we will use them to discover venues near our neighbourhoods

In [23]:
CLIENT_ID = 'KUWOR5KHKEMA5XQG4P0HHY10OJTFWTH30NUVROXI0BA4DROV' # your Foursquare ID
CLIENT_SECRET = 'WF3DJ2UHZSB3HH41RQ5ECHXOVZTGGWZLR42F0SKNQGLGWYOE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KUWOR5KHKEMA5XQG4P0HHY10OJTFWTH30NUVROXI0BA4DROV
CLIENT_SECRET:WF3DJ2UHZSB3HH41RQ5ECHXOVZTGGWZLR42F0SKNQGLGWYOE


function to get nearby venues of all neighbourhoods

In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Getting venues nearby neighbourhoods of toronto

In [46]:
toronto_venues = getNearbyVenues(names=torontoData['Neighbourhood'],
                                   latitudes=torontoData['Latitude'],
                                   longitudes=torontoData['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

In [82]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 234 uniques categories.


Converting it into one hot format 

In [47]:
toronto_onehot=pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

In [48]:
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Grouping neighbourhoods to find mean occurance of venue categories

In [52]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

Getting 10 most common venues categories of neighbourhoods

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [61]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Steakhouse,Asian Restaurant,Restaurant,Hotel,Thai Restaurant,Cosmetics Shop,Gym
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Steakhouse,Bakery,Farmers Market,Café,Cheese Shop,Beer Bar,Liquor Store
2,"Brockton,Exhibition Place,Parkdale Village",Bakery,Coffee Shop,Café,Breakfast Spot,Gym,Performing Arts Venue,Pet Store,Nightclub,Climbing Gym,Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Park,Garden,Light Rail Station,Farmers Market,Spa,Fast Food Restaurant,Burrito Place,Restaurant,Brewery,Auto Workshop
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Bar,Harbor / Marina


Clustering neighbourhoods on the basis of venue categories occurance 

In [67]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [69]:
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       4, 0, 1, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Merging toronto data with most common venues

In [71]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = torontoData

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,latitude_geocode,longitude_geocode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.671024,-79.296712,43.676357,-79.293031,0,Pub,Other Great Outdoors,Neighborhood,Health Food Store,Trail,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant
41,M4K,East Toronto,"The Danforth West,Riverdale",40.820312,-87.977818,43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Yoga Studio,Sports Bar,Spa,Juice Bar,Brewery
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.671024,-79.296712,43.668999,-79.315572,0,Park,Pizza Place,Brewery,Burger Joint,Burrito Place,Sandwich Place,Pub,Coffee Shop,Gym,Sushi Restaurant
43,M4M,East Toronto,Studio District,-12.271035,49.291897,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,Bakery,Gastropub,American Restaurant,Brewery,Latin American Restaurant,Fish Market,Ice Cream Shop
44,M4N,Central Toronto,Lawrence Park,42.152279,-80.023112,43.728020,-79.388790,4,Park,Swim School,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [74]:
!conda install -c conda-forge folium=0.5.0

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         673 KB

The following NEW packages will be INSTALLED:

    altair:  4.0.1-py_0 conda-forge
    branca:  0.3.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


folium-0.5.0         | 45 KB     | ##################################### | 1

Showing clusters of neighbourhoods on map using folium

In [77]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 0

In [84]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,43.676357,-79.293031,0,Pub,Other Great Outdoors,Neighborhood,Health Food Store,Trail,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant
41,East Toronto,43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Yoga Studio,Sports Bar,Spa,Juice Bar,Brewery
42,East Toronto,43.668999,-79.315572,0,Park,Pizza Place,Brewery,Burger Joint,Burrito Place,Sandwich Place,Pub,Coffee Shop,Gym,Sushi Restaurant
43,East Toronto,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,Bakery,Gastropub,American Restaurant,Brewery,Latin American Restaurant,Fish Market,Ice Cream Shop
45,Central Toronto,43.712751,-79.390197,0,Gym,Food & Drink Shop,Sandwich Place,Department Store,Asian Restaurant,Hotel,Dog Run,Breakfast Spot,Park,Eastern European Restaurant
46,Central Toronto,43.715383,-79.405678,0,Sporting Goods Shop,Coffee Shop,Yoga Studio,Spa,Diner,Restaurant,Rental Car Location,Salon / Barbershop,Dessert Shop,Metro Station
47,Central Toronto,43.704324,-79.388790,0,Dessert Shop,Sandwich Place,Gym,Sushi Restaurant,Italian Restaurant,Pizza Place,Café,Coffee Shop,Brewery,Toy / Game Store
49,Central Toronto,43.686412,-79.400049,0,Pub,Coffee Shop,American Restaurant,Liquor Store,Sports Bar,Restaurant,Supermarket,Sushi Restaurant,Fried Chicken Joint,Light Rail Station
51,Downtown Toronto,43.667967,-79.367675,0,Coffee Shop,Restaurant,Italian Restaurant,Bakery,Pizza Place,Café,Pub,Gastropub,Caribbean Restaurant,Indian Restaurant
52,Downtown Toronto,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Gym,Pub,Men's Store,Mediterranean Restaurant,Hotel


Cluster 1

In [85]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,43.689574,-79.38316,1,Trail,Tennis Court,Yoga Studio,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


Cluster 2

In [87]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,43.679563,-79.377529,2,Park,Playground,Trail,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
64,Central Toronto,43.696948,-79.411307,2,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


Cluster 3

In [88]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,43.711695,-79.416936,3,Pool,Garden,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


Cluster 4

In [89]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,43.72802,-79.38879,4,Park,Swim School,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
